In [7]:
!pip install matplotlib
!pip install tensorflow==2.8.0
!pip install tensorflow-io==0.25.0

  Using cached tensorflow_io-0.25.0-cp310-cp310-win_amd64.whl.metadata (14 kB)
  Using cached tensorflow_io_gcs_filesystem-0.25.0-cp310-cp310-win_amd64.whl.metadata (14 kB)
Using cached tensorflow_io-0.25.0-cp310-cp310-win_amd64.whl (21.8 MB)
Using cached tensorflow_io_gcs_filesystem-0.25.0-cp310-cp310-win_amd64.whl (1.5 MB)
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.31.0
    Uninstalling tensorflow-io-gcs-filesystem-0.31.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.31.0


In [3]:
!pip install "numpy<2"

   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ----------- ---------------------------- 4.5/15.8 MB 24.4 MB/s eta 0:00:01
   --------------- ------------------------ 6.3/15.8 MB 24.1 MB/s eta 0:00:01
   ---------------------------- ----------- 11.3/15.8 MB 19.6 MB/s eta 0:00:01
   ---------------------------------------  15.7/15.8 MB 20.6 MB/s eta 0:00:01
   ---------------------------------------- 15.8/15.8 MB 19.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.0
    Uninstalling numpy-2.2.0:
      Successfully uninstalled numpy-2.2.0


  You can safely remove it manually.
  You can safely remove it manually.


In [1]:
!conda list

# packages in environment at C:\Users\foxir\miniconda3\envs\gunshot_tf:
#
# Name                    Version                   Build  Channel
absl-py                   2.1.0                    pypi_0    pypi
asttokens                 3.0.0                    pypi_0    pypi
astunparse                1.6.3                    pypi_0    pypi
bzip2                     1.0.8                h2466b09_7    conda-forge
ca-certificates           2024.12.14           h56e8100_0    conda-forge
cachetools                5.5.0                    pypi_0    pypi
certifi                   2024.12.14               pypi_0    pypi
charset-normalizer        3.4.0                    pypi_0    pypi
colorama                  0.4.6                    pypi_0    pypi
comm                      0.2.2                    pypi_0    pypi
contourpy                 1.3.1                    pypi_0    pypi
cudatoolkit               11.2.2              h7d7167e_13    conda-forge
cudnn                     8.1.0.77            

In [4]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio
import numpy as np

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

In [11]:
!python -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
data = os.path.join('belize-data')
train_dir = os.path.join('belize-data', 'Training data')
val_dir = os.path.join('belize-data', 'Validation data')
val_NEG = os.path.join('belize-data', 'Validation data', 'Background')
val_POS = os.path.join('belize-data', 'Validation data', 'Gunshot')
train_NEG = os.path.join('belize-data', 'Training data', 'Background')
train_POS = os.path.join('belize-data', 'Training data', 'Gunshot')
model_save_dir = os.path.join('model','model_exports')
outputs_dir = os.path.join('outputs')

In [10]:
pos = tf.data.Dataset.list_files(train_POS+'/*.wav')
neg = tf.data.Dataset.list_files(train_NEG+'/*.wav')

positives = tf.data.Dataset.zip((pos, tf.data.Dataset.from_tensor_slices(tf.ones(len(pos)))))
negatives = tf.data.Dataset.zip((neg, tf.data.Dataset.from_tensor_slices(tf.zeros(len(neg)))))
training_data = positives.concatenate(negatives)

In [13]:
test_pos = tf.data.Dataset.list_files(val_POS+'/*.wav')
test_neg = tf.data.Dataset.list_files(val_NEG+'/*.wav')

test_positives = tf.data.Dataset.zip((test_pos, tf.data.Dataset.from_tensor_slices(tf.ones(len(test_pos)))))
test_negatives = tf.data.Dataset.zip((test_neg, tf.data.Dataset.from_tensor_slices(tf.ones(len(test_neg)))))
testing_data = positives.concatenate(negatives)

In [14]:
data = training_data
print(f"Label shape1: {labels.shape}")
data = data.map(preprocess_mel)
print(f"Label shape2: {labels.shape}")
data = data.cache()
print(f"Label shape3: {labels.shape}")
data = data.shuffle(buffer_size=1000)
print(f"Label shape4: {labels.shape}")
data = data.batch(16)
print(f"Label shape5: {labels.shape}")
data = data.prefetch(8)
print(f"Label shape6: {labels.shape}")

NameError: name 'labels' is not defined

In [ ]:
testing_data = testing_data.map(preprocess_mel)
testing_data = testing_data.cache()
print(f"Label shape7: {labels.shape}")

In [ ]:
train = data.take(36)
test = data.skip(36).take(15)
print(f"Label shape1: {labels.shape}")
samples, labels = train.as_numpy_iterator().next()
print(f"Label shape2: {labels.shape}")

In [ ]:
labels = labels.reshape(-1, 1)

In [ ]:
print(f"Label shape3: {labels.shape}")

In [12]:
def load_wav_8k_mono(filename):
    try:
        file_contents = tf.io.read_file(filename)
        wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
        sample_rate = tf.cast(sample_rate, dtype=tf.int64)
        if sample_rate != 8000:
            wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=8000)
        wav = tf.squeeze(wav, axis=-1)
    except Exception as e:
        print(f"Invalid file: {file_path} - {e}")
    return wav

In [ ]:
sample_rate = 8000
mel_spec_length = 32000
frame_length = 256
frame_step = 128
mel_bins = 64
upper_edge = 4000
lower_edge = 0

In [ ]:
def preprocess_mel(file_path, label):
    # load wav and make sure length is proper
    try:
        wav = load_wav_8k_mono(file_path)
    except Exception as e:
        print(f"Failed to load wav file at {file_path} - {e}")
    if len(wav) > 32000:
        wav = wav[:mel_spec_length]
    if len(wav) < 32000:
        try:
            zero_padding = tf.zeros([mel_spec_length] - tf.shape(wav), dtype=tf.float32)
            wav = tf.concat([zero_padding, wav], 0)
        except Exception as e:
            print(f"Failed to pad wav file at {file_path} - {e}")

    # create the spectrogram
    print("Wav Shape:", wav.shape)
    spectrogram = tf.signal.stft(wav, frame_length, frame_step, window_fn=tf.signal.hann_window)
    print("STFT Shape1:", spectrogram.shape)
    spectrogram = tf.abs(spectrogram)
    print("STFT Shape2:", spectrogram.shape)

    # create the mel filterbank
    spec_bins = frame_length // 2 + 1  # Derived from frame length
    mel_filterbank = tf.signal.linear_to_mel_weight_matrix(
        mel_bins,
        spec_bins,
        sample_rate,
        lower_edge,
        upper_edge,
    )
    print("Mel Filterbank Shape:", mel_filterbank.shape)
    
    # create the mel spectrogram
    mel_spectrogram = tf.matmul(spectrogram, mel_filterbank)
    print("Mel Spectrogram Shape1:", mel_spectrogram.shape)

    # add a dimension
    mel_spectrogram = tf.expand_dims(mel_spectrogram, axis=2)
    print("Mel Spectrogram Shape2:", mel_spectrogram.shape)

    return mel_spectrogram, label

In [ ]:
def preprocess(file_path, label):
    # load wav and make sure length is proper
    wav = load_wav_8k_mono(file_path)
    # wav = wav[:mel_spec_length]
    # zero_padding = tf.zeros([mel_spec_length] - tf.shape(wav), dtype=tf.float32)
    # wav = tf.concat([zero_padding, wav],0)
    # create the spectrogram
    spectrogram = tf.signal.stft(wav, frame_length, frame_step)
    spectrogram = tf.abs(spectrogram)
    # add a dimension
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram, label